In [63]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs


In [64]:
%pip install --quiet openai
from openai import OpenAI

In [65]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [66]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [121]:
import os

from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')


In [68]:

from typing import Tuple, List, Optional

In [69]:

from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [70]:

from langchain_core.runnables import ConfigurableField

In [71]:

from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [72]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [73]:

from langchain_community.vectorstores import Neo4jVector

In [74]:

from langchain_community.graphs import Neo4jGraph

In [108]:
NEO4J_URI="neo4j+s://082f2916.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="1NKby4KeZHHfu9yaIjC-4UJQWT15DjmFIMOihNNfDCc"

In [122]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [110]:
%pip install --quiet -U langchain-neo4j


In [79]:
from langchain_neo4j import Neo4jGraph
graph = Neo4jGraph()

In [111]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="List of presidents of the United States").load()

In [38]:
len(raw_documents)

25

In [39]:
raw_documents[:3]

[Document(metadata={'title': 'List of presidents of the United States', 'summary': 'The president of the United States is the head of state and head of government of the United States, indirectly elected to a four-year term via the Electoral College. Under the U.S. Constitution, the officeholder leads the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces. The first president, George Washington, won a unanimous vote of the Electoral College. The incumbent president is Donald Trump, who assumed office on January 20, 2025. Since the office was established in 1789, 45 men have served in 47 presidencies; the discrepancy arises from two individuals elected to non-consecutive terms: Grover Cleveland is counted as the 22nd and 24th president of the United States, while Donald Trump is counted as the 45th and 47th president.\nThe presidency of William Henry Harrison, who died 31 days after taking office in 1841, was the shortest in Americ

In [40]:
raw_documents[0]

Document(metadata={'title': 'List of presidents of the United States', 'summary': 'The president of the United States is the head of state and head of government of the United States, indirectly elected to a four-year term via the Electoral College. Under the U.S. Constitution, the officeholder leads the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces. The first president, George Washington, won a unanimous vote of the Electoral College. The incumbent president is Donald Trump, who assumed office on January 20, 2025. Since the office was established in 1789, 45 men have served in 47 presidencies; the discrepancy arises from two individuals elected to non-consecutive terms: Grover Cleveland is counted as the 22nd and 24th president of the United States, while Donald Trump is counted as the 45th and 47th president.\nThe presidency of William Henry Harrison, who died 31 days after taking office in 1841, was the shortest in America

# Tokenizing the data

In [81]:

from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

In [82]:
len(documents)

6

In [125]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

In [126]:

from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [127]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [128]:
graph_documents

[GraphDocument(nodes=[Node(id='United States', type='Country', properties={}), Node(id='President', type='Position', properties={}), Node(id='Electoral College', type='Institution', properties={}), Node(id='George Washington', type='Person', properties={}), Node(id='Donald Trump', type='Person', properties={}), Node(id='Grover Cleveland', type='Person', properties={}), Node(id='William Henry Harrison', type='Person', properties={}), Node(id='Franklin D. Roosevelt', type='Person', properties={}), Node(id='Zachary Taylor', type='Person', properties={}), Node(id='Warren G. Harding', type='Person', properties={}), Node(id='Abraham Lincoln', type='Person', properties={}), Node(id='James A. Garfield', type='Person', properties={}), Node(id='William Mckinley', type='Person', properties={}), Node(id='John F. Kennedy', type='Person', properties={}), Node(id='Richard Nixon', type='Person', properties={}), Node(id='John Tyler', type='Person', properties={}), Node(id='Alexander Hamilton', type='Pe

In [129]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [130]:
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [131]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [132]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [133]:

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [134]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))